# Setup

In [1]:
from db_funcs import get_all_content, get_db_con, insert_article, insert_question
from utils import Sentence_Extractor
from neo4j_funcs import Neo4j_Driver
from EntityRelationshipExtractor import ER_Extractor
from tqdm import tqdm
import requests
import dotenv
from my_qdrant import MyQdrant
from sentence_transformers import SentenceTransformer
import os

dotenv.load_dotenv()

neo4j = Neo4j_Driver()
sentence_extractor = Sentence_Extractor()
er_extractor=ER_Extractor()

/home/chris/miniconda3/envs/GraphRAG/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to /home/chris/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Device set to use cuda:0


# SQL Database Creation
Fills a sql database with the content from the squad 2.0 Dataset

In [4]:
# Download SQuAD 2.0 dev dataset
url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json'
response = requests.get(url)
squad_dev = response.json()

conn=get_db_con()

for topic in squad_dev["data"]:
    print(topic['title'])
    for paragraph in topic["paragraphs"]:
        with conn.cursor() as cur:
            article_id = insert_article(cur, topic["title"], paragraph["context"])
            for qa in paragraph["qas"]:
                insert_question(cur, qa["id"], article_id, qa["question"], qa["answers"], qa["is_impossible"])
        conn.commit()

Normans
Computational_complexity_theory
Southern_California
Sky_(United_Kingdom)
Victoria_(Australia)
Huguenot
Steam_engine
Oxygen
1973_oil_crisis
European_Union_law
Amazon_rainforest
Ctenophora
Fresno,_California
Packet_switching
Black_Death
Geology
Pharmacy
Civil_disobedience
Construction
Private_school
Harvard_University
Jacksonville,_Florida
Economic_inequality
University_of_Chicago
Yuan_dynasty
Immune_system
Intergovernmental_Panel_on_Climate_Change
Prime_number
Rhine
Scottish_Parliament
Islamism
Imperialism
Warsaw
French_and_Indian_War
Force


# Embedding Ingestion
Creates a bunch of embeddings for each of the content blocks and stores it in the qdrant embedding database alongside a content_id

In [2]:
with get_db_con().cursor() as cur:
    content_ids, all_content = get_all_content(cur)

model = SentenceTransformer("all-MiniLM-L6-v2")
qdrant_client=MyQdrant(os.getenv('QDRANT_COLLECTION_NAME'),os.getenv('QDRANT_VECTOR_SIZE'))

print("Creating embeddings from content")
content_embeddings=model.encode(all_content)

print("adding embeddings to db")
qdrant_client.add_points(content_embeddings, content_ids)

print("embedding db is ready")

Skipping creating collection; 'graphRAG' already exists.
Creating embeddings from content
adding embeddings to db
embedding db is ready


# Extract Sentances and Entity Relationships
For each content block from the database, extract all sentences from the content block. For each content block, extract all entities and relationships using the model and system defined in the entity relationship extractor. Finally, insert each extracted entity into the neo4j database keeping track of which content block the entity is from. Entities can be from multiple content blocks so neo4j stores an array of content ids for each entity node.

In [6]:

with get_db_con().cursor() as cur:
    content_ids, all_content = get_all_content(cur)
for content_id, content in tqdm(zip(content_ids, all_content), total=len(content_ids), desc="Processing content"):
    sentences = sentence_extractor.get_sentences(content)
    triplets=er_extractor.extract_ERs(sentences)
    for e in triplets:
        neo4j.insert_relationship(e.head, "entity", e.type, e.tail, "entity", content_id)
    

Processing content:  40%|███▉      | 479/1204 [10:41<16:11,  1.34s/it]


KeyboardInterrupt: 